# EAS Integration Tests

We use this notebook to test EAS integration branches on three different platforms:

- Juno
- Hikey 620
- TC2

These are basic tests to check that EAS-related features are in place. It is assumed that the kernel has been built for each platform and has correctly booted. Tests are the following:

1. Printing kernel version
+ All CPUs are up and running
+ All sched domains are present
+ The energy model is present
+ Check that the proper sched domain flags are enabled
+ EAS-related config options are there
+ cpufreq is present
+ cpuidle is present
+ utilest is available among the SCHED_FEATURES

In [ ]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=logging.INFO)

In [ ]:
%matplotlib inline

# Support to access the remote target
import devlib
from env import TestEnv

import pandas as pd

# Expected Data

In [ ]:
configs = [
    'CONFIG_ARM_BIG_LITTLE_CPUFREQ',
    'CONFIG_CGROUPS',
    'CONFIG_CPU_FREQ',
    'CONFIG_CPU_FREQ_STAT',
    'CONFIG_CPU_FREQ_GOV_SCHEDUTIL',
    'CONFIG_CPU_FREQ_DEFAULT_GOV_SCHEDUTIL',
    'CONFIG_SCHED_DEBUG',
    'CONFIG_FUNCTION_TRACER',
    'CONFIG_IRQSOFF_TRACER',
    'CONFIG_CPU_IDLE',
    'CONFIG_SCHEDSTATS',
    'CONFIG_SCHED_TRACER',
    'CONFIG_SCHED_MC',
    'CONFIG_FTRACE',
    'CONFIG_CGROUPS',
    'CONFIG_CGROUP_FREEZER',
]

In [ ]:
energy_model = {
    'juno': {
        # sched domains (on CPU0)
        0: {
            # sched groups (on CPU0)
            0: {
                'cap_states': [(276, 37), (501, 59), (593, 117)],
                'idle_states': [33, 33, 0, 0]
            },
            1: {
                'cap_states': [(276, 37), (501, 59), (593, 117)],
                'idle_states': [33, 33, 0, 0]
            },
            2: {
                'cap_states': [(276, 37), (501, 59), (593, 117)],
                'idle_states': [33, 33, 0, 0]
            },
            3: {
                'cap_states': [(276, 37), (501, 59), (593, 117)],
                'idle_states': [33, 33, 0, 0]
            },
        },
        1: {
            0: {
                'cap_states': [(276, 41), (501, 86), (593, 107)],
                'idle_states': [41, 41, 41, 14]
            },
            1: {
                'cap_states': [(501, 48), (849, 73), (1024, 107)],
                'idle_states': [48, 48, 48, 18]
            }
        }
    },
    'hikey': {
        0: {
            0: {
                'cap_states': [(178, 69), (369, 125), (622, 224), (819, 367), (1024, 670)],
                'idle_states': [15, 15, 0, 0]
            },
            1: {
                'cap_states': [(178, 69), (369, 125), (622, 224), (819, 367), (1024, 670)],
                'idle_states': [15, 15, 0, 0]
            },
            2: {
                'cap_states': [(178, 69), (369, 125), (622, 224), (819, 367), (1024, 670)],
                'idle_states': [15, 15, 0, 0]
            },
            3: {
                'cap_states': [(178, 69), (369, 125), (622, 224), (819, 367), (1024, 670)],
                'idle_states': [15, 15, 0, 0]
            },
        },
        1: {
            0: {
                'cap_states': [(178, 16), (369, 29), (622, 47), (819, 75), (1024, 112)],
                'idle_states': [107, 107, 47, 0]
            },
            1: {
                'cap_states': [(178, 16), (369, 29), (622, 47), (819, 75), (1024, 112)],
                'idle_states': [107, 107, 47, 0]
            },            
        },
        2: {
            0: {
                'cap_states': [(1024, 0)],
                'idle_states': [0, 0, 0, 0]
            },
        },
    },
    'tc2': {
        0: {
            0: {
                'cap_states': [(426, 2021), (512, 2312), (597, 2756), (682, 3125),
                               (768, 3524), (853, 3846), (938, 5177), (1024, 6997)],
                'idle_states': [0, 0, 0]
            },
            1: {
                'cap_states': [(426, 2021), (512, 2312), (597, 2756), (682, 3125),
                               (768, 3524), (853, 3846), (938, 5177), (1024, 6997)],
                'idle_states': [0, 0, 0]
            },
        },
        1: {
            0: {
                'cap_states': [(426, 7920), (512, 8165), (597, 8172), (682, 8195),
                               (768, 8265), (853, 8446), (938, 11426), (1024, 15200)],
                'idle_states': [70, 70, 25]
            },
            1: {
                'cap_states': [(150, 2967), (172, 2792), (215, 2810), (258, 2815),
                               (301, 2919), (344, 2847), (387, 3917), (430, 4905)],
                'idle_states': [25, 25, 10]
            },            
        },
    },
    'hikey960': {
        0: {
            0: {
                'cap_states': [(133, 87), (250, 164), (351, 265), (429, 388), (462, 502)],
                'idle_states': [5, 5, 0, 0]
            },
            1: {
                'cap_states': [(133, 87), (250, 164), (351, 265), (429, 388), (462, 502)],
                'idle_states': [5, 5, 0, 0]
            },
            2: {
                'cap_states': [(133, 87), (250, 164), (351, 265), (429, 388), (462, 502)],
                'idle_states': [5, 5, 0, 0]
            },
            3: {
                'cap_states': [(133, 87), (250, 164), (351, 265), (429, 388), (462, 502)],
                'idle_states': [5, 5, 0, 0]
            },
        },
        1: {
            0: {
                'cap_states': [(133, 12), (250, 22), (351, 36), (429, 67), (462, 144)],
                'idle_states': [12, 12, 12, 0]
            },
            1: {
                'cap_states': [(390, 102), (615, 124), (782, 221), (915, 330), (1024, 433)],
                'idle_states': [102, 102, 102, 102, 0]
            }
        }
    },
}

In [ ]:
sched_domains = {
    'juno': ['MC', 'DIE'],
    'hikey': ['SYS', 'MC', 'DIE'],
    'tc2': ['MC', 'DIE'],
    'hikey960': ['MC', 'DIE'],
}

In [ ]:
cpus = {
    'juno': ['0xd03', '0xd08', '0xd08', '0xd03', '0xd03', '0xd03'],
    'hikey': ['0xd03', '0xd03', '0xd03', '0xd03', '0xd03', '0xd03', '0xd03', '0xd03'],
    'tc2': ['0xc0f', '0xc0f', '0xc07', '0xc07', '0xc07'],
    'hikey960': ['0xd03', '0xd03', '0xd03', '0xd03', '0xd09', '0xd09', '0xd09', '0xd09'],
}

In [ ]:
# Sched domain flags
SD_LOAD_BALANCE        = 0x0001
SD_BALANCE_NEWIDLE     = 0x0002
SD_BALANCE_EXEC        = 0x0004
SD_BALANCE_FORK        = 0x0008
SD_BALANCE_WAKE        = 0x0010
SD_WAKE_AFFINE         = 0x0020
SD_ASYM_CPUCAPACITY    = 0x0040
SD_SHARE_CPUCAPACITY   = 0x0080
SD_SHARE_POWERDOMAIN   = 0x0100
SD_SHARE_PKG_RESOURCES = 0x0200
SD_SERIALIZE           = 0x0400
SD_ASYM_PACKING        = 0x0800
SD_PREFER_SIBLING      = 0x1000
SD_OVERLAP             = 0x2000
SD_NUMA                = 0x4000
SD_SHARE_CAP_STATES    = 0x8000

In [ ]:
# Sched domain flags for big.LITTLE platforms
sd_flags_bl = {
    0: (SD_LOAD_BALANCE | SD_BALANCE_NEWIDLE | SD_WAKE_AFFINE | SD_PREFER_SIBLING | SD_BALANCE_EXEC |
        SD_BALANCE_FORK | SD_BALANCE_WAKE | SD_SHARE_PKG_RESOURCES | SD_SHARE_CAP_STATES),
    1: (SD_LOAD_BALANCE | SD_BALANCE_NEWIDLE | SD_WAKE_AFFINE | SD_PREFER_SIBLING | SD_BALANCE_EXEC |
        SD_BALANCE_FORK | SD_ASYM_CPUCAPACITY | SD_BALANCE_WAKE)
}

sd_flags_smp = {
    0: (SD_LOAD_BALANCE | SD_BALANCE_NEWIDLE | SD_WAKE_AFFINE | SD_PREFER_SIBLING | SD_BALANCE_EXEC |
        SD_BALANCE_FORK | SD_SHARE_PKG_RESOURCES),
    1: (SD_LOAD_BALANCE | SD_BALANCE_NEWIDLE | SD_WAKE_AFFINE | SD_PREFER_SIBLING | SD_BALANCE_EXEC |
        SD_BALANCE_FORK | SD_SHARE_CAP_STATES),
    2: (SD_WAKE_AFFINE | SD_BALANCE_NEWIDLE | SD_PREFER_SIBLING | SD_BALANCE_EXEC | SD_BALANCE_FORK),
}

# Expected sched domain flags
sd_flags = {
    'juno': sd_flags_bl,
    'hikey': sd_flags_smp,
    'tc2': sd_flags_bl,
    'hikey960': sd_flags_bl,
}

In [ ]:
# Number of frequency domains in each platform
# This should correspond to the number of cpufreq policies
frequency_domains = {
    'juno': 2,
    'hikey': 1,
    'tc2': 2,
    'hikey960': 2,
}

In [ ]:
idle_states = {
    'juno': 3,
    'hikey': 3,
    'tc2': 2,
    'hikey960': 3,
}

# Test Methods

In [ ]:
W  = '\033[0m'  # white (normal)
R  = '\033[31m' # red
G  = '\033[32m' # green

In [ ]:
def check_config(te, platform): 
    print "Kernel Config"
    print "==========="
    
    config_path = '/proc/config.gz'
    if not te.target.file_exists(config_path):
        print R + "FAILURE" + W
        print "Missing {} file".format(config_path)
        print "Is the option enabled in the config?"
        return
    
    all_enabled = True
    for c in configs:
        if not te.target.config.is_enabled(c):
            all_enabled = False
            print "{} option NOT ENABLED".format(c)

    if all_enabled:
        print G + "OK" + W
    else:
        print R + "FAILURE" + W
        print "Missing options in .config"
    print ""

In [ ]:
def check_sched_domains(te, platform):
    print "Sched Domains"
    print "============="
    sd = te.target.execute('cat /proc/sys/kernel/sched_domain/cpu0/domain*/name').split()
    if set(sd) == set(sched_domains[platform]):
        print G + "OK" + W
    else:
        print R + "FAILURE" + W
        print "Some sched domaind do not correspond to the expected ones"
        print "Expected: {}".format(sched_domains[platform])
        print "Read from target: {}".format(sd)
    print ""

In [ ]:
def check_cpu_parts(te, platform):
    print "CPU parts"
    print "============"
    c = te.target.execute('cat /proc/cpuinfo | grep "^CPU part" | cut -d" " -f3').split('\r\n')[:-1]
    if c == cpus[platform]:
        print G + "OK" + W
    else:
        print R + "FAILURE" + W
        print "Some CPU parts do not correspond to the expected ones", "red"
        print "Expected: {}".format(cpus[platform]), "red"
        print "Read from target: {}".format(c)
    print ""

In [ ]:
def check_energy_model(te, platform):
    print "Energy Model"
    print "============"
    nrg_model = energy_model[platform]
    error = False
    for sd in nrg_model:
        for sg in nrg_model[sd]:
            cap_states = te.target.execute(
                'cat /proc/sys/kernel/sched_domain/cpu0/domain{}/group{}/energy/cap_states'.format(sd, sg)
            ).split()
            cap_states = [int(c) for c in cap_states]
            cap_states = zip(cap_states[::2], cap_states[1::2])
            idle_states = te.target.execute(
                'cat /proc/sys/kernel/sched_domain/cpu0/domain{}/group{}/energy/idle_states'.format(sd, sg)
            ).split()
            idle_states = [int(i) for i in idle_states]
            if cap_states != nrg_model[sd][sg]['cap_states']:
                print "Sched domain {}, sched group {} shows wrong cap_states".format(sd, sg)
                print "Expected: {}".format(nrg_model[sd][sg]['cap_states'])
                print "Read from target: {}".format(cap_states)
                error = True
            if idle_states != nrg_model[sd][sg]['idle_states']:
                print "Sched domain {}, sched group {} shows wrong idle_states".format(sd, sg)
                print "Expected: {}".format(nrg_model[sd][sg]['idle_states'])
                print "Read from target: {}".format(idle_states)
                error = True

    if error:
        print R + "FAILURE" + W
    else:
        print G + "OK" + W
    print ""

In [ ]:
def check_sched_domain_flags(te, platform):
    ta_flags = te.target.execute('cat /proc/sys/kernel/sched_domain/cpu0/domain*/flags').split()
    ta_flags = [int(f) for f in ta_flags]

    print "Sched Domain Flags"
    print "============"
    error = False
    for i, flag in enumerate(ta_flags):
        if sd_flags[platform][i] != flag:
            error = True
            print "Sched Domain {}".format(i)
            print "Expected {}, read {}".format(sd_flags[platfom][i], flag)

    if error:
        print R + "FAILURE" + W
    else:
        print G + "OK" + W
    print ""

In [ ]:
def check_cpufreq_policies(te, platform):
    print "cpufreq Policies"
    print "==========="
    n_policies = int(te.target.execute('ls -l /sys/devices/system/cpu/cpufreq/ | grep policy | wc -l'))
    if n_policies == frequency_domains[platform]:
        print G + "OK" + W
    else:
        print R + "FAILURE" + W
        print "Number of cpufreq policies does not match number of frequency domains"
        print "Expected: {}".format(frequency_domains[platform])
        print "Read from target: {}".format(n_policies)
    print ""

In [ ]:
def check_cpuidle_states(te, platform):
    print "cpuidle States"
    print "==========="
    states = int(te.target.execute('cat /sys/devices/system/cpu/cpu0/cpuidle/state*/name | wc -l'))
    if states == idle_states[platform]:
        print G + "OK" + W
    else:
        print R + "FAILURE" + W
        print "Wrong number of idle states"
        print "Expected: {}".format(idle_states[platform])
        print "Read from target: {}".format(states)
    print ""

In [ ]:
def check_utilest(te, platform):
    print "UtilEst"
    print "==========="
    sched_features = te.target.execute('cat /sys/kernel/debug/sched_features | grep UTIL_EST', as_root=True)
    if not sched_features:
        print R + "FAILURE" + W
        print "Sched feature UtilEst not available"
    else:
        print G + "OK" + W
    print ""

In [ ]:
def test_eas_integration(te, platform):
    """
    Print test results for EAS integration branch.
    
    :param platform: test platform name. Valid values:
        - hikey
        - juno
        - tc2
        - hikey960
    :type platform: str
    
    :param te: test environment object
    :type te: env.TestEnv
    """
    platform = platform.lower()
    available_platforms = ['hikey', 'juno', 'tc2', 'hikey960']
    if platform not in available_platforms:
        raise ValueError('Invalid platform value {}, accepted values are: {}'
                         .format(available_platforms))

    print "Kernel Version"
    print "=============="
    print te.target.kernel_version
    print ""

    check_config(te, platform)
    check_sched_domains(te, platform)
    check_cpu_parts(te, platform)
    check_energy_model(te, platform)
    check_sched_domain_flags(te, platform)
    check_cpufreq_policies(te, platform)
    check_cpuidle_states(te, platform)
    check_utilest(te, platform)

# Juno

In [ ]:
# Where results will be collected
RESULTS_BASE = 'eas_integration/juno'

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'linux',
    "board"        : 'juno-r2',

    "host": "10.1.210.22",
    
    "modules" : ['cpufreq', 'cgroups'],
    
    "results_dir" : RESULTS_BASE,
}

In [ ]:
te = TestEnv(my_conf, force_new=True)

In [ ]:
test_eas_integration(te, 'juno')

# Hikey620

In [ ]:
# Where results will be collected
RESULTS_BASE = 'eas_integration/hikey620'

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'linux',
    "board"        : 'hikey',

    "host": "10.1.210.62",
    
    "modules" : ['cpufreq', 'cgroups'],
    
    "results_dir" : RESULTS_BASE,
}

In [ ]:
te = TestEnv(my_conf, force_new=True)

In [ ]:
test_eas_integration(te, 'hikey')

# TC2

In [ ]:
# Where results will be collected
RESULTS_BASE = 'eas_integration/tc2'

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'linux',
    "board"        : 'tc2',

    "host": "10.1.210.72",
    
    "modules" : ['cpufreq', 'cgroups', 'cpuidle'],
    
    "results_dir" : RESULTS_BASE,
}

In [ ]:
te = TestEnv(my_conf, force_new=True)

In [ ]:
test_eas_integration(te, 'tc2')

# Hikey960

In [ ]:
# Where results will be collected
RESULTS_BASE = 'eas_integration/hikey960'

In [ ]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'linux',
    "board"        : 'hikey960',
    "username"     : 'linaro',
    "password"     : 'linaro',

    "host": "10.1.210.44",
    
    "modules" : ['cpufreq', 'cgroups'],
    
    "results_dir" : RESULTS_BASE,
}

In [ ]:
te = TestEnv(my_conf, force_new=True)

In [ ]:
test_eas_integration(te, 'hikey960')

# Patch Statistics

In [ ]:
import os

from datetime import datetime

stats_filename = 'stats_{}.txt'.format(datetime.now().strftime('%Y%m%d'))

KERNEL_GIT_DIR = '/data/IntegrationBase/kernel/.git/'
MANIFEST_DIR   = '/data/IntegrationBase/kernel.manifest/'
MANIFEST_FILE  = os.path.join(MANIFEST_DIR, 'manifest')
STATS_FILE     = os.path.join(MANIFEST_DIR, 'patch_stats', stats_filename)

In [ ]:
import re
import subprocess

re_topic = re.compile("topic\s+(\S+)\s+(\S+)\s+TIP\s+\-\s+rebase\s+(\S+)")


f = open(MANIFEST_FILE, 'r')
fout = open(STATS_FILE, 'w')

total_commits = 0


fout.write("### Integration patch statistics\n")
fout.write("### manifest: {}\n\n".format(MANIFEST_FILE))
fout.write("{:30} {:8} {:2} {:40}\n".format("### topic","owner","#","branch"))

for line in f:
    topic_def = re_topic.match(line)
    if not (topic_def):
        continue

    topic = topic_def.group(1)
    base = topic_def.group(3)
    tip = topic_def.group(2)

    owner = tip.split('/')[0]

    commits = subprocess.check_output(["git",
                                       "--git-dir", KERNEL_GIT_DIR,
                                       "log",
                                       "--oneline",
                                       base+".."+tip])
    patch_count = len(commits.splitlines())
    total_commits += patch_count
    fout.write("{:30} {:8} {:2} {:40}\n".format(topic,owner,patch_count,tip))

fout.write("{:30} {:8} {:2} {:40}\n".format("### Total","",total_commits,""))

f.close()
fout.close()

In [ ]:
!cat $STATS_FILE